# SNN-IDS: Pipeline di Intrusion Detection con Federated Learning e Homomorphic Encryption

Questo notebook è la guida principale e interattiva per il progetto SNN-IDS. L'obiettivo è fornire una spiegazione chiara e riproducibile delle scelte architetturali e tecniche, permettendo di rieseguire l'intero flusso di lavoro, dalla preparazione dei dati al training di modelli complessi come quelli federati e sicuri.

**Obiettivi del Notebook:**
1.  **Trasparenza:** Documentare ogni fase del processo.
2.  **Riproducibilità:** Consentire di rieseguire gli esperimenti e ottenere risultati coerenti.
3.  **Modularità:** Interagire con il codice sorgente, strutturato come una libreria (`snn_ids`), senza duplicare la logica.

**Struttura del Notebook:**
*   **Parte 1: Setup e Configurazione:** Preparazione dell'ambiente e analisi del file di configurazione centrale.
*   **Parte 2: La Pipeline dei Dati:** Caricamento, preprocessing e caching.
*   **Parte 3: Esperimento - Training Centralizzato (Baseline):** Addestramento di un modello standard per avere un riferimento.
*   **Parte 4: Esperimento - Federated Learning (FL):** Simulazione di un addestramento federato.
*   **Parte 5: Esperimento - Secure FL con Homomorphic Encryption (HE):** Aggiunta di un livello di privacy con la crittografia omomorfica.
*   **Parte 6: Valutazione e Confronto:** Analisi dei risultati dei vari esperimenti.
*   **Parte 7: Benchmarking Automatizzato:** Come usare lo script `benchmark.py` per test su larga scala.

## Parte 1: Setup e Configurazione

### 1.1 Installazione delle Dipendenze

Assicuriamoci che tutte le librerie necessarie siano installate. Il file `requirements.txt` contiene la lista completa.

In [ ]:
!pip install -q -r requirements.txt

### 1.2 Import delle Librerie e Moduli del Progetto

Importiamo le librerie standard e i moduli principali dal nostro package `snn_ids`.

In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Aggiungiamo la directory 'src' al path per permettere l'import del nostro package
sys.path.append(os.path.abspath('src'))

from config import DATA_CONFIG, PREPROCESSING_CONFIG, TRAINING_CONFIG, FEDERATED_CONFIG, HOMOMORPHIC_CONFIG
from snn_ids.preprocessing.process import preprocess_pipeline

print("Librerie e moduli caricati con successo!")

### 1.3 Analisi della Configurazione Centrale (`config.py`)

Tutto il comportamento della pipeline è controllato da un unico file, `config.py`. Questo approccio centralizzato (Single Source of Truth) evita di avere parametri "magici" sparsi nel codice e rende gli esperimenti facili da configurare e riprodurre.

Esaminiamo le sezioni principali:

In [ ]:
print("--- CONFIGURAZIONE DATI ---")
print(json.dumps(DATA_CONFIG, indent=2))

print("\n--- CONFIGURAZIONE PREPROCESSING ---")
print(json.dumps(PREPROCESSING_CONFIG, indent=2))

print("\n--- CONFIGURAZIONE FEDERATED LEARNING ---")
print(json.dumps(FEDERATED_CONFIG, indent=2))

print("\n--- CONFIGURAZIONE HOMOMORPHIC ENCRYPTION ---")
print(json.dumps(HOMOMORPHIC_CONFIG, indent=2))

## Parte 2: La Pipeline dei Dati

### 2.1 Caching dei Dati

Il preprocessing dei dati è un'operazione costosa. Per ottimizzare il processo, il nostro sistema utilizza due livelli di cache:

1.  **`preprocessed_cache`**: Contiene i file CSV originali già parzialmente puliti e con le classi bilanciate. Questo evita di dover rileggere e filtrare i file sorgente ad ogni esecuzione.
2.  **`model_cache`**: Contiene i dati finali pronti per il training (array NumPy `X` e `y`, e il `LabelEncoder`). Questo è il livello di cache più veloce e viene usato per accelerare esperimenti ripetuti con la stessa configurazione di dati.

Per questo notebook, useremo la `model_cache` che è già presente nel repository.

### 2.2 Esecuzione del Preprocessing

Eseguiamo la pipeline di preprocessing. Questa funzione caricherà i dati direttamente dalla `model_cache` se la trova.

In [ ]:
X, y, label_encoder = preprocess_pipeline(
    sample_size=10000 # Questo valore corrisponde a quello della model_cache
)

print(f"Preprocessing completato.")
print(f"Forma di X: {X.shape}")
print(f"Forma di y: {y.shape}")
print(f"Numero di classi: {len(label_encoder.classes_)}")

## Parte 3: Esecuzione tramite Runner Unificato

Tutta la logica per l'esecuzione dei diversi tipi di esperimenti è stata consolidata nel runner `benchmark.py`. Questo script permette di lanciare workflow complessi con un singolo comando.

Qui sotto mostriamo come eseguire i principali workflow direttamente dal notebook.

### 3.1 Esperimento Centralizzato (Smoke Test)

In [ ]:
!python3 benchmark.py centralized --smoke-test --sample-size 1000

### 3.2 Esperimento Federato

In [ ]:
!python3 benchmark.py federated --sample-size 1000

### 3.3 Esperimento Federato con Homomorphic Encryption

In [ ]:
!python3 benchmark.py federated --he --sample-size 1000